In [42]:

from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 15 trillion tokens model 2x faster!
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # We also uploaded 4bit for 405b!
    "unsloth/Mistral-Nemo-Base-2407-bnb-4bit", # New Mistral 12b 2x faster!
    "unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit",
    "unsloth/mistral-7b-v0.3-bnb-4bit",        # Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
    )

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2025-01-09 11:21:52.821303: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-09 11:21:52.835517: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1736389312.852979  676581 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1736389312.858348  676581 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-09 11:21:52.877424: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

Unsloth: OpenAI failed to import - ignoring for now.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2024.12.4: Fast Llama patching. Transformers:4.46.3.
   \\   /|    GPU: NVIDIA A100-PCIE-40GB. Max memory: 39.394 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.0+cu124. CUDA: 8.0. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post2. FA2 = True]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [43]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

# alpaca_prompt = You MUST copy from above!
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""


inputs = tokenizer(
[
    alpaca_prompt.format(
        "Recommend a daily diet that includes a specific ingredient.",

        "Create a diet that includes baby bok choy(roots trimmed and roughly chopped).",

        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 2048)

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Recommend a daily diet that includes a specific ingredient.

### Input:
Create a diet that includes baby bok choy(roots trimmed and roughly chopped).

### Response:


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


I recommend a daily diet that includes baby bok choy(roots trimmed and roughly chopped).<|end_of_text|>


In [49]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer
import torch

# 데이터프레임 로드 (예제)
df = df[(df['task']=='daily_diets')| (df['task']=='alternative_diet')]

# 빈 output 및 nutrition 컬럼 추가
df["output"] = ""
df["nutrition_info"] = ""

# Alpaca 스타일 프롬프트 설정
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request. If the output includes a diet, ensure to include detailed nutritional information (calories, sugar, sodium, fiber, protein, fat) for the diet as well.

### Instruction:
{}

### Input:
{}

### Response:
"""

# 모델을 사용해 output 및 영양소 정보 생성
def generate_output_and_nutrition(row):
    prompt = alpaca_prompt.format(row["instruction"], row["input"])
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    # Text Streamer 사용하여 결과 생성
    text_streamer = TextStreamer(tokenizer)
    output = model.generate(**inputs, max_new_tokens=200, streamer=text_streamer)

    # 생성된 텍스트 디코딩 및 처리
    decoded_output = tokenizer.decode(output[0], skip_special_tokens=True)

    # "Diet:"와 "Nutrition:" 구분
    if "Nutrition:" in decoded_output:
        diet_part, nutrition_part = decoded_output.split("Nutrition:", 1)
        diet = diet_part.strip().split("### Response:")[-1].strip()
        nutrition = nutrition_part.strip()
    else:
        diet = decoded_output.split("### Response:")[-1].strip()
        nutrition = "Nutrition information not provided."

    return diet, nutrition

# 데이터프레임에 output 및 nutrition 정보 생성 후 저장
for index, row in df.iterrows():
    diet, nutrition = generate_output_and_nutrition(row)
    df.at[index, "output"] = diet
    df.at[index, "nutrition_info"] = nutrition

# 결과 확인 및 저장
# df.to_csv("output_with_diet_and_nutrition.csv", index=False)
# print("Output saved to 'output_with_diet_and_nutrition.csv'")


<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request. If the output includes a diet, ensure to include detailed nutritional information (calories, sugar, sodium, fiber, protein, fat) for the diet as well.

### Instruction:
Recommend a daily diet based on the given nutritional goals.

### Input:
Ensure the daily carbohydrate intake does not exceed 71g, protein intake is at least 58g, and fat intake does not exceed 44g.

### Response:
1. The daily carbohydrate intake does not exceed 71g.
2. The protein intake is at least 58g.
3. The fat intake does not exceed 44g.

### Output:
1. A diet with 71g of carbohydrates, 58g of protein, and 44g of fat.
2. A diet with 71g of carbohydrates, 58g of protein, and 44g of fat.
3. A diet with 71g of carbohydrates, 58g of protein, and 44g of fat.

### Explanation:
The diet should be designed to meet the given nutritional goals while al

In [52]:
df['output'].iloc[1]

'A diet that includes Parmesan cheese(grated, for the topping) could consist of the following:\n\n| Meal | Ingredient | Serving Size | Calories | Sugar | Sodium | Fiber | Protein | Fat |\n| --- | --- | --- | --- | --- | --- | --- | --- | --- |\n| Breakfast | Pancakes | 1 serving | 400 | 40 | 300 | 5 | 20 | 20 |\n| Breakfast | Grated Parmesan cheese | 1 serving | 100 | 10 | 50 | 5 | 5 | 5 |\n| Lunch | Grilled chicken | 1 serving | 200 | 10 | 100 | 0 | 30 | 5 |\n| Lunch | Grated Parmesan cheese | 1 serving | 100 | 10 | 50 | 5 | 5 | 5 |\n| Dinner | Pasta | 1 serving | 300 | 10 | 100 | 5 |'

In [50]:
df.head()

,dataset,split_data,task,instruction,input,output,__index_level_0__,model_output,nutrition_info
430,diabetes_food_hub,test,daily_diets,Recommend a daily diet based on the given nutr...,Ensure the daily carbohydrate intake does not ...,1. The daily carbohydrate intake does not exce...,32166,"{""Breakfast"": ""Turkey Sausage and Egg Casserol...",Nutrition information not provided.
431,diabetes_food_hub,test,daily_diets,Recommend a daily diet that includes a specifi...,Create a diet that includes Parmesan cheese(gr...,"A diet that includes Parmesan cheese(grated, f...",32167,"{""Breakfast"": ""Turkey Meatball \u201cWonton\u2...",Nutrition information not provided.
432,diabetes_food_hub,test,daily_diets,Recommend a daily diet that includes a specifi...,Create a diet that includes olive oil.,A diet that includes olive oil should be balan...,32168,"{""Breakfast"": ""Turkey Meatball \u201cWonton\u2...",Nutrition information not provided.
433,diabetes_food_hub,test,daily_diets,Recommend a daily diet that includes a specifi...,Create a diet that includes butternut squash(f...,A diet that includes butternut squash for the ...,32169,"{""Breakfast"": ""Turkey Meatball \u201cWonton\u2...",Nutrition information not provided.
434,diabetes_food_hub,test,daily_diets,Analyze and summarize the nutritional content ...,"{""Breakfast"": ""Chicken and Cucumber Lettuce Wr...",* Breakfast: Chicken and Cucumber Lettuce Wrap...,32170,"{""Daily Total"": {""Carbohydrate (g)"": 60, ""Prot...",Nutrition information not provided.


In [59]:
import pandas as pd
import openai
from statistics import mean
from dotenv import load_dotenv
import os

# 환경 변수 로드
load_dotenv(dotenv_path="/data/jaesung/llm_for_diabetes/.env")
openai.api_key = os.getenv("OPENAI_API_KEY")

df = df

# Nutri-score 계산 함수
def calculate_nutri_score(calories, sugar, sodium, fiber, fat, protein):
    # Negative points
    negative_points = 0
    if calories > 800:
        negative_points += 10
    elif calories > 720:
        negative_points += 9
    elif calories > 640:
        negative_points += 8
    elif calories > 560:
        negative_points += 7
    elif calories > 480:
        negative_points += 6
    elif calories > 400:
        negative_points += 5
    elif calories > 320:
        negative_points += 4
    elif calories > 240:
        negative_points += 3
    elif calories > 160:
        negative_points += 2
    elif calories > 80:
        negative_points += 1

    if sugar > 45:
        negative_points += 10
    elif sugar > 40:
        negative_points += 9
    elif sugar > 36:
        negative_points += 8
    elif sugar > 31:
        negative_points += 7
    elif sugar > 27:
        negative_points += 6
    elif sugar > 22.5:
        negative_points += 5
    elif sugar > 18:
        negative_points += 4
    elif sugar > 13.5:
        negative_points += 3
    elif sugar > 9:
        negative_points += 2
    elif sugar > 4.5:
        negative_points += 1

    if fat > 10:
        negative_points += 10
    elif fat > 9:
        negative_points += 9
    elif fat > 8:
        negative_points += 8
    elif fat > 7:
        negative_points += 7
    elif fat > 6:
        negative_points += 6
    elif fat > 5:
        negative_points += 5
    elif fat > 4:
        negative_points += 4
    elif fat > 3:
        negative_points += 3
    elif fat > 2:
        negative_points += 2
    elif fat > 1:
        negative_points += 1

    if sodium > 900:
        negative_points += 10
    elif sodium > 810:
        negative_points += 9
    elif sodium > 720:
        negative_points += 8
    elif sodium > 630:
        negative_points += 7
    elif sodium > 540:
        negative_points += 6
    elif sodium > 450:
        negative_points += 5
    elif sodium > 360:
        negative_points += 4
    elif sodium > 270:
        negative_points += 3
    elif sodium > 180:
        negative_points += 2
    elif sodium > 90:
        negative_points += 1

    # Positive points
    positive_points = 0
    if fiber > 4.7:
        positive_points += 5
    elif fiber > 3.5:
        positive_points += 4
    elif fiber > 2.8:
        positive_points += 3
    elif fiber > 2.1:
        positive_points += 2
    elif fiber > 1.4:
        positive_points += 1

    if protein > 8:
        positive_points += 5
    elif protein > 6.4:
        positive_points += 4
    elif protein > 4.8:
        positive_points += 3
    elif protein > 3.2:
        positive_points += 2
    elif protein > 1.6:
        positive_points += 1

    # Nutri-score 계산
    nutri_score = negative_points - positive_points
    return nutri_score

# GPT를 통해 영양소 데이터 추론 함수
def infer_nutritional_data(output):
    prompt = (
        f"Please estimate the nutritional values for the following dietary recommendation. Return only numerical values without units or text for calories, sugar, sodium, fiber, fat, and protein:\n\n"
        f"Diet Recommendation:\n"
        f"{output}\n\n"
        f"Return the result in the format:\n"
        f"Calories: <value>\n"
        f"Sugar: <value>\n"
        f"Sodium: <value>\n"
        f"Fiber: <value>\n"
        f"Fat: <value>\n"
        f"Protein: <value>\n"
    )
    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-0125",
            messages=[{"role": "system", "content": "You are a helpful assistant."},
                      {"role": "user", "content": prompt}],
            max_tokens=150,
            temperature=0.5
        )
        evaluation = response["choices"][0]["message"]["content"]
        values = {}
        for line in evaluation.split("\n"):
            if "Calories" in line:
                values["calories"] = float(line.split(":")[1].strip())
            elif "Sugar" in line:
                values["sugar"] = float(line.split(":")[1].strip())
            elif "Sodium" in line:
                values["sodium"] = float(line.split(":")[1].strip())
            elif "Fiber" in line:
                values["fiber"] = float(line.split(":")[1].strip())
            elif "Fat" in line:
                values["fat"] = float(line.split(":")[1].strip())
            elif "Protein" in line:
                values["protein"] = float(line.split(":")[1].strip())
        return values
    except Exception as e:
        print(f"Error during GPT inference: {e}")
        return {"calories": 0.0, "sugar": 0.0, "sodium": 0.0, "fiber": 0.0, "fat": 0.0, "protein": 0.0}

# GPT를 통해 coherence, completeness, naturalness 평가 요청
def evaluate_output(output):
    prompt = (
        f"Please evaluate the following dietary recommendation on the following criteria:\n\n"
        f"1. **Coherence** (1-5): How logically and meaningfully connected the meals are. "
        f"Rate 1 if the meals are completely disconnected, and 5 if they are logically consistent and flow well.\n"
        f"2. **Completeness** (1-5): How complete the meal plan is, considering all meals and balance. "
        f"Rate 1 if essential components are missing, and 5 if all elements are balanced and present.\n"
        f"3. **Naturalness** (1-5): How practical and realistic the meal plan sounds for daily consumption. "
        f"Rate 1 if it is impractical, and 5 if it is highly practical and realistic.\n\n"
        f"Diet Recommendation:\n"
        f"{output}\n\n"
        f"Return the result in the exact format:\n"
        f"Coherence: <score>\n"
        f"Completeness: <score>\n"
        f"Naturalness: <score>\n"
    )
    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-0125",
            messages=[{"role": "system", "content": "You are a helpful assistant."},
                      {"role": "user", "content": prompt}],
            max_tokens=150,
            temperature=0.7
        )
        evaluation = response["choices"][0]["message"]["content"]
        scores = {}
        for line in evaluation.split("\n"):
            if "Coherence" in line:
                scores["Coherence"] = float(line.split(":")[1].strip())
            elif "Completeness" in line:
                scores["Completeness"] = float(line.split(":")[1].strip())
            elif "Naturalness" in line:
                scores["Naturalness"] = float(line.split(":")[1].strip())
        return scores
    except Exception as e:
        print(f"Error during GPT evaluation: {e}")
        return {"Coherence": None, "Completeness": None, "Naturalness": None}

# 데이터프레임에 평가 점수 추가
df["Coherence"] = None
df["Completeness"] = None
df["Naturalness"] = None
df["Nutri-score"] = None

for index, row in df.iterrows():
    # Coherence, Completeness, Naturalness 계산
    scores = evaluate_output(row["output"])
    df.at[index, "Coherence"] = scores.get("Coherence", None)
    df.at[index, "Completeness"] = scores.get("Completeness", None)
    df.at[index, "Naturalness"] = scores.get("Naturalness", None)

    # Nutri-score 계산을 위한 GPT 영양소 추론
    nutrition = infer_nutritional_data(row["output"])
    calories = nutrition.get("calories", 0.0)
    sugar = nutrition.get("sugar", 0.0)
    sodium = nutrition.get("sodium", 0.0)
    fiber = nutrition.get("fiber", 0.0)
    fat = nutrition.get("fat", 0.0)
    protein = nutrition.get("protein", 0.0)

    nutri_score = calculate_nutri_score(calories, sugar, sodium, fiber, fat, protein)
    df.at[index, "Nutri-score"] = nutri_score

# 평균 점수 계산
def calculate_averages(df):
    averages = {
        "Coherence Average": df["Coherence"].mean(),
        "Completeness Average": df["Completeness"].mean(),
        "Naturalness Average": df["Naturalness"].mean(),
        "Nutri-score Average": df["Nutri-score"].mean()
    }
    return averages

averages = calculate_averages(df)
print("Average Scores:", averages)

# 결과 저장
# df.to_csv("evaluated_meal_plans.csv", index=False)
# print("Evaluation saved to 'evaluated_meal_plans.csv'")


Error during GPT inference: could not convert string to float: ''
Error during GPT inference: could not convert string to float: '1,600'
Error during GPT inference: could not convert string to float: 'Not provided'
Error during GPT inference: could not convert string to float: 'Not provided'
Error during GPT inference: could not convert string to float: ''
Error during GPT inference: could not convert string to float: ''
Error during GPT inference: could not convert string to float: '104g'
Average Scores: {'Coherence Average': 3.816326530612245, 'Completeness Average': 3.3979591836734695, 'Naturalness Average': 3.4591836734693877, 'Nutri-score Average': 9.96}


In [16]:
import json

file_path = "/data/jaesung/llm_for_diabetes/src/model/inference_results_0114.jsonl"

data = []
with open(file_path, 'r', encoding='utf-8') as f:
    for line in f:
        data.append(json.loads(line))

In [17]:
data

[{'dataset': 'diabetes_food_hub',
  'split_data': 'test',
  'task': 'daily_diets',
  'instruction': 'Recommend a daily diet based on the given nutritional goals.',
  'input': 'Ensure the daily carbohydrate intake does not exceed 108.0g, protein intake is at least 36.0g, and fat intake does not exceed 39.0g.',
  'output': '{"Breakfast": "Sun-Kissed Health Salad", "Lunch": "Greek Bean and Vegetable Soup", "Dinner": "Curried Chickpea Stew with Roasted Vegetables"}',
  '__index_level_0__': 28967,
  'model_output': '{"Breakfast": "Maple-Pumpkin Spice Oatmeal Cookies", "Lunch": "Turkey Meatball \\u201cWonton\\u201d Soup with Bok Choy & Carrots", "Dinner": "Curried Chickpea Stew with Roasted Vegetables"}'},
 {'dataset': 'diabetes_food_hub',
  'split_data': 'test',
  'task': 'daily_diets',
  'instruction': 'Recommend a daily diet based on the given nutritional goals.',
  'input': 'Ensure the daily carbohydrate intake does not exceed 88.0g, protein intake is at least 36.0g, and fat intake does 

In [18]:
import pandas as pd

df = pd.DataFrame(data)

In [19]:
df.head(2)

,dataset,split_data,task,instruction,input,output,__index_level_0__,model_output
0,diabetes_food_hub,test,daily_diets,Recommend a daily diet based on the given nutr...,Ensure the daily carbohydrate intake does not ...,"{""Breakfast"": ""Sun-Kissed Health Salad"", ""Lunc...",28967,"{""Breakfast"": ""Maple-Pumpkin Spice Oatmeal Coo..."
1,diabetes_food_hub,test,daily_diets,Recommend a daily diet based on the given nutr...,Ensure the daily carbohydrate intake does not ...,"{""Breakfast"": ""Tofu and Vegetable Skewers"", ""L...",27867,"{""Breakfast"": ""Peach & Cream Smoothie"", ""Lunch..."


In [5]:
df['task'].unique()

array(['daily_diets', 'qa_descriptive', 'alternative_diet',
       'ie_extract_relation', 'qa_subjective', 'qa_objective', 'nli',
       'summarization', 'generation'], dtype=object)

In [21]:
# qa_objective

import pandas as pd
qa_objective_df = df[df['task']=='qa_objective']

# 'output'와 'model_output'에서 A), B), C), D)만 추출
qa_objective_df['output_label'] = qa_objective_df['output'].str.extract(r'(A\)|B\)|C\)|D\))')
qa_objective_df['model_output_label'] = qa_objective_df['model_output'].str.extract(r'(A\)|B\)|C\)|D\))')

# 두 컬럼 비교하여 맞은 경우를 계산
qa_objective_df['correct'] = qa_objective_df['output_label'] == qa_objective_df['model_output_label']

# Accuracy 계산
accuracy = qa_objective_df['correct'].mean()

print(f"Accuracy: {accuracy:.2%}")


Accuracy: 40.00%


/tmp/ipykernel_2949341/2713378175.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  qa_objective_df['output_label'] = qa_objective_df['output'].str.extract(r'(A\)|B\)|C\)|D\))')
/tmp/ipykernel_2949341/2713378175.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  qa_objective_df['model_output_label'] = qa_objective_df['model_output'].str.extract(r'(A\)|B\)|C\)|D\))')
/tmp/ipykernel_2949341/2713378175.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try usi

In [22]:
import pandas as pd
qa_subjective_df = df[df['task']=='qa_subjective']

# 'output'와 'model_output'에서 A), B), C), D)만 추출
qa_subjective_df['output_label'] = qa_subjective_df['output'].str.extract(r'(A\)|B\)|C\)|D\))')
qa_subjective_df['model_output_label'] = qa_subjective_df['model_output'].str.extract(r'(A\)|B\)|C\)|D\))')

# 두 컬럼 비교하여 맞은 경우를 계산
qa_subjective_df['correct'] = qa_subjective_df['output_label'] == qa_subjective_df['model_output_label']

# Accuracy 계산
accuracy = qa_subjective_df['correct'].mean()

print(f"Accuracy: {accuracy:.2%}")

Accuracy: 46.00%


/tmp/ipykernel_2949341/1884220316.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  qa_subjective_df['output_label'] = qa_subjective_df['output'].str.extract(r'(A\)|B\)|C\)|D\))')
/tmp/ipykernel_2949341/1884220316.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  qa_subjective_df['model_output_label'] = qa_subjective_df['model_output'].str.extract(r'(A\)|B\)|C\)|D\))')
/tmp/ipykernel_2949341/1884220316.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try 

In [26]:
# qa_descriptive

def token_overlap(output, model_output):
    # 토큰화
    output_tokens = set(output.split())
    model_output_tokens = set(model_output.split())
    
    # 공통 토큰 개수 계산
    common_tokens = output_tokens.intersection(model_output_tokens)
    
    # 일치 비율 계산
    return len(common_tokens) / len(output_tokens) if len(output_tokens) > 0 else 0

qa_descriptive_df = df[df['task'] == 'qa_descriptive']

# 일치 비율 계산 및 저장
qa_descriptive_df['token_match_score'] = qa_descriptive_df.apply(lambda row: token_overlap(row['output'], row['model_output']), axis=1)

# 평균 점수를 정확도로 간주
accuracy = qa_descriptive_df['token_match_score'].mean()

print(f"Token Match Accuracy: {accuracy:.2%}")


Token Match Accuracy: 28.26%


/tmp/ipykernel_2949341/308410962.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  qa_descriptive_df['token_match_score'] = qa_descriptive_df.apply(lambda row: token_overlap(row['output'], row['model_output']), axis=1)


In [24]:
# nli

nli_df = df[df['task'] == 'nli']

correct_predictions = (nli_df['output'] == nli_df['model_output']).sum()
total_predictions = len(nli_df) 

nli_acc = correct_predictions / total_predictions

print(nli_acc)  # bioinstruct - 0.33 

0.54


In [27]:
df[df['task'] == 'ie_extract_relation']

,dataset,split_data,task,instruction,input,output,__index_level_0__,model_output
150,biorel,test,ie_extract_relation,Predict the relationship between the given ent...,long-term treatment of acromegaly with bromocr...,may_be_treated_by,26117,has_parent
151,biorel,test,ie_extract_relation,Predict the relationship between the given ent...,"at glucose concentrations above 3.3 mm , cytop...",has_physical_part_of_anatomic_structure,26150,gene_product_of_gene
152,biorel,test,ie_extract_relation,Predict the relationship between the given ent...,the activities of glucose-6-phosphate dehydrog...,disease_has_primary_anatomic_site,26088,may_treat
153,biorel,test,ie_extract_relation,Predict the relationship between the given ent...,urinary insulin-to-creatinine clearance ratio ...,may_treat,26111,may_treat
154,biorel,test,ie_extract_relation,Predict the relationship between the given ent...,in order to evaluate the proportions of somato...,tissue_is_expression_site_of_gene_product,26017,biological_process_has_physical_quantity
155,biorel,test,ie_extract_relation,Predict the relationship between the given ent...,no donors had a history of diabetes or pancrea...,disease_has_associated_anatomic_site,26102,anatomic_structure_has_anatomic_structure
156,biorel,test,ie_extract_relation,Predict the relationship between the given ent...,when comparing values over time with pretreatm...,tissue_is_expression_site_of_gene_product,26045,is_associated_anatomic_site_of
157,biorel,test,ie_extract_relation,Predict the relationship between the given ent...,dietary change among adult dogrib indians of t...,conceptual_part_of,25985,anatomic_structure_pointed_to_by_anatomic_stru...
158,biorel,test,ie_extract_relation,Predict the relationship between the given ent...,the function of plasma membrane as control poi...,anatomic_structure_is_physical_part_of,26113,anatomic_structure_is_physical_part_of
159,biorel,test,ie_extract_relation,Predict the relationship between the given ent...,in physiologically controlled puppies ( bp 66 ...,has_physical_part_of_anatomic_structure,26037,gene_product_is_target_of


In [25]:
# ie - ie_extract_relation
from sklearn.metrics import precision_score, recall_score, f1_score


ie_extract_relation = df[df['task'] == 'ie_extract_relation']

y_true = ie_extract_relation['output']
y_pred = ie_extract_relation['model_output']

precision = precision_score(y_true, y_pred, average='weighted')
recall = recall_score(y_true, y_pred, average='weighted')
f1 = f1_score(y_true, y_pred, average='weighted')

print(f"precision: {precision}") 
print(f"recall: {recall}")
print(f"f1: {f1}")


precision: 0.08666666666666666
recall: 0.06
f1: 0.07


/data/jaesung/anaconda3/envs/meal/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/data/jaesung/anaconda3/envs/meal/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
# summarization

import openai
import pandas as pd
import re
from sklearn.metrics import f1_score
from bert_score import score as bert_score
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

import os
from dotenv import load_dotenv

load_dotenv()

openai.api_key = os.getenv("OPENAI_API_KEY")

bleurt_model_name = "Elron/bleurt-large-512"
tokenizer = AutoTokenizer.from_pretrained(bleurt_model_name)
bleurt_model = AutoModelForSequenceClassification.from_pretrained(bleurt_model_name)
bleurt_model.eval()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
bleurt_model = bleurt_model.to(device)

MAX_CONTEXT_LENGTH = 8192

def evaluate_with_gpt4(input_text, model_output, true_output):

    input_text = input_text[:MAX_CONTEXT_LENGTH]
    model_output = model_output[:MAX_CONTEXT_LENGTH]
    true_output = true_output[:MAX_CONTEXT_LENGTH]

    prompt = f"""
    You are tasked with evaluating the quality of a QA model's responses based on the following metrics:
    1. **Coherence**: Does the model's response logically align with the context provided in the input?
    2. **Completeness**: Does the model's response sufficiently answer the question in the input?
    3. **Naturalness**: Does the model's response sound fluent and human-like?

    **Input**:
    {input_text}

    **Model's Response**:
    {model_output}

    **True Answer**:
    {true_output}

    Please rate each metric on a scale from 1 to 5. 
    Example response format:
    - Coherence: X.X
    - Completeness: X.X
    - Naturalness: X.X
    """
    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-0125",
            messages=[
                {"role": "system", "content": "You are an expert evaluator for Summarization models."},
                {"role": "user", "content": prompt}
            ]
        )
        return response["choices"][0]["message"]["content"]
    except Exception as e:
        print("Error with GPT-4 API:", e)
        return None

def extract_scores(evaluation):

    if evaluation is None:
        return {
            "Coherence": 0.0,
            "Completeness": 0.0,
            "Naturalness": 0.0
        }
    coherence = re.search(r"Coherence: (\d\.\d)", evaluation)
    completeness = re.search(r"Completeness: (\d\.\d)", evaluation)
    naturalness = re.search(r"Naturalness: (\d\.\d)", evaluation)
    
    return {
        "Coherence": float(coherence.group(1)) if coherence else 0.0,
        "Completeness": float(completeness.group(1)) if completeness else 0.0,
        "Naturalness": float(naturalness.group(1)) if naturalness else 0.0
    }

def calculate_concept_f1(y_true, y_pred):
    return f1_score(y_true, y_pred, average="micro")

def calculate_bleurt(y_true, y_pred):

    inputs = tokenizer(y_pred, y_true, return_tensors="pt", padding=True, truncation=True, max_length=512)
    inputs = {key: value.to(device) for key, value in inputs.items()}
    
    with torch.no_grad():
        scores = bleurt_model(**inputs).logits

    if scores.numel() == 1:
        return float(scores.squeeze().item())  
    return [float(score) for score in scores.squeeze().tolist()]  

def calculate_bleurt_and_bertscore(y_true, y_pred):

    bleurt_score_value = calculate_bleurt(y_true, y_pred)

    _, _, bert_f1 = bert_score(y_pred, y_true, lang="en", rescale_with_baseline=True)

    bert_f1 = [float(score) for score in bert_f1]

    return {
        "BLEURT": bleurt_score_value if isinstance(bleurt_score_value, float) else sum(bleurt_score_value) / len(bleurt_score_value),
        "BERTScore_F1": sum(bert_f1) / len(bert_f1)
    }

def normalize_scores(df, column):
    df[column] = df[column].apply(
        lambda x: float(x) if isinstance(x, torch.Tensor) else x
    )
    min_val = df[column].min()
    max_val = df[column].max()
    df[column] = df[column].apply(
        lambda x: (x - min_val) / (max_val - min_val) if max_val > min_val else 0.5
    )
    return df

qa_df = df[df['task'] == 'summarization']

results = []

for _, row in qa_df.iterrows():
    input_text = row['input']
    model_output = row['model_output']
    true_output = row['output']
    
    evaluation = evaluate_with_gpt4(input_text, model_output, true_output)
    print(f"Evaluation result:\n{evaluation}")
    
    scores = extract_scores(evaluation)
    
    concept_f1 = calculate_concept_f1([true_output], [model_output])
    
    metric_scores = calculate_bleurt_and_bertscore([true_output], [model_output])

    results.append({
        "input": input_text,
        "model_output": model_output,
        "true_output": true_output,
        "evaluation": evaluation,
        "Coherence": scores["Coherence"],
        "Completeness": scores["Completeness"],
        "Naturalness": scores["Naturalness"],
        "Concept_F1": concept_f1,
        "BLEURT": metric_scores["BLEURT"],
        "BERTScore_F1": metric_scores["BERTScore_F1"]
    })

evaluation_df = pd.DataFrame(results)

evaluation_df = normalize_scores(evaluation_df, "BLEURT")
evaluation_df = normalize_scores(evaluation_df, "BERTScore_F1")

average_scores = evaluation_df[["Coherence", "Completeness", "Naturalness", "Concept_F1", "BLEURT", "BERTScore_F1"]].mean()

print("평균 점수:")
print(average_scores)

Evaluation result:
- Coherence: 4.0
- Completeness: 3.5
- Naturalness: 4.0


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Evaluation result:
- Coherence: 4.5
- Completeness: 4.0
- Naturalness: 4.5


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Evaluation result:
- Coherence: 3.5
- Completeness: 3.0
- Naturalness: 2.5


...

Evaluation result:
- Coherence: 4.5
- Completeness: 4.0
- Naturalness: 3.5


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Evaluation result:
- Coherence: 4.5
- Completeness: 4.0
- Naturalness: 4.0


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


평균 점수:
Coherence       3.850000
Completeness    3.433333
Naturalness     3.483333
Concept_F1      0.000000
BLEURT          0.539845
BERTScore_F1    0.603795
dtype: float64


In [8]:
import re
from sklearn.metrics import accuracy_score

def extract_choice(text):
    match = re.search(r'\b([A-D])\b', text)
    return match.group(1) if match else None

df['output_parsed'] = df['output'].apply(extract_choice)
df['model_output_parsed'] = df['model_output'].apply(extract_choice)

filtered_df = df.dropna(subset=['output_parsed', 'model_output_parsed'])

accuracy = accuracy_score(filtered_df['output_parsed'], filtered_df['model_output_parsed'])

print(f"Accuracy: {accuracy:.4f}")


Accuracy: 0.4100


In [ ]:
# daily diets

import openai
import pandas as pd
import re
from tqdm import tqdm

from dotenv import load_dotenv
import os

load_dotenv()

openai.api_key = os.getenv("OPENAI_API_KEY")

MAX_CONTEXT_LENGTH = 8192

def evaluate_with_gpt4(input_text, model_output, true_output, task_type):
    input_text = input_text[:MAX_CONTEXT_LENGTH]
    model_output = model_output[:MAX_CONTEXT_LENGTH]
    true_output = true_output[:MAX_CONTEXT_LENGTH]

    if task_type == "daily_diets":
        prompt = f"""
        You are tasked with evaluating the quality of a meal recommendation model's responses based on the following metrics:
        1. **Coherence**: Does the model's response logically align with the context provided in the input?
        2. **Completeness**: Does the model's response sufficiently answer the input request?
        3. **Naturalness**: Does the model's response sound fluent and human-like?
        4. **Nutritional Adequacy**: Does the meal align with the nutritional goals in the input, considering reasonable flexibility and practical applicability in real-life scenarios?
        5. **Caloric Balance**: Are the recommended meals well-balanced in terms of calories?

        **Input**:
        {input_text}

        **Model's Response**:
        {model_output}

        **True Answer**:
        {true_output}

        Please rate each metric on a scale from 1 to 5. 
        Example response format:
        - Coherence: X.X
        - Completeness: X.X
        - Naturalness: X.X
        - Nutritional Adequacy: X.X
        - Caloric Balance: X.X
        """
    elif task_type == "alternative_diets":
        prompt = f"""
        You are tasked with evaluating the quality of a meal recommendation model's responses based on the following metrics:
        1. **Coherence**: Does the model's response logically align with the context provided in the input?
        2. **Completeness**: Does the model's response sufficiently answer the input request?
        3. **Naturalness**: Does the model's response sound fluent and human-like?
        4. **Improvement**: Does the recommended meal address the shortcomings of the previous meal?
        5. **Suitability**: Is the recommended meal suitable for a diabetes patient?

        **Input**:
        {input_text}

        **Model's Response**:
        {model_output}

        **True Answer**:
        {true_output}

        Please rate each metric on a scale from 1 to 5. 
        Example response format:
        - Coherence: X.X
        - Completeness: X.X
        - Naturalness: X.X
        - Improvement: X.X
        - Suitability: X.X
        """
    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-0125",
            messages=[
                {"role": "system", "content": "You are an expert evaluator for meal recommendation models."},
                {"role": "user", "content": prompt}
            ]
        )
        return response["choices"][0]["message"]["content"]
    except Exception as e:
        print("Error with GPT-4 API:", e)
        return None

# 점수 추출 함수
def extract_scores(evaluation, task_type):
    if evaluation is None:
        if task_type == "daily_diets":
            return {
                "Coherence": 0.0,
                "Completeness": 0.0,
                "Naturalness": 0.0,
                "Nutritional Adequacy": 0.0,
                "Caloric Balance": 0.0
            }
        elif task_type == "alternative_diets":
            return {
                "Coherence": 0.0,
                "Completeness": 0.0,
                "Naturalness": 0.0,
                "Improvement": 0.0,
                "Suitability": 0.0
            }

    scores = {}
    if task_type == "daily_diets":
        metrics = ["Coherence", "Completeness", "Naturalness", "Nutritional Adequacy", "Caloric Balance"]
    elif task_type == "alternative_diets":
        metrics = ["Coherence", "Completeness", "Naturalness", "Improvement", "Suitability"]

    for metric in metrics:
        match = re.search(fr"{metric}: (\d\.\d)", evaluation)
        scores[metric] = float(match.group(1)) if match else 0.0

    return scores

results = []

daily_df = df[(df['task'] == 'daily_diets') & (df['output'].str.contains('"Breakfast"'))]

for _, row in tqdm(daily_df.iterrows(), total=len(daily_df), desc="Evaluating daily diets"):
    evaluation = evaluate_with_gpt4(row['input'], row['model_output'], row['output'], "daily_diets")
    scores = extract_scores(evaluation, "daily_diets")
    results.append({**row.to_dict(), **scores})

alternative_df = df[df['task'] == 'alternative_diets']

for _, row in tqdm(alternative_df.iterrows(), total=len(alternative_df), desc="Evaluating alternative diets"):
    evaluation = evaluate_with_gpt4(row['input'], row['model_output'], row['output'], "alternative_diets")
    scores = extract_scores(evaluation, "alternative_diets")
    results.append({**row.to_dict(), **scores})

evaluation_df = pd.DataFrame(results)

if "Nutritional Adequacy" in evaluation_df.columns and "Caloric Balance" in evaluation_df.columns:
    daily_avg = evaluation_df[evaluation_df['task'] == 'daily_diets'][[
        "Coherence", "Completeness", "Naturalness", 
        "Nutritional Adequacy", "Caloric Balance"
    ]].mean()
    print("Daily Diets Average Scores:")
    print(daily_avg)

if "Improvement" in evaluation_df.columns and "Suitability" in evaluation_df.columns:
    alternative_avg = evaluation_df[evaluation_df['task'] == 'alternative_diets'][[
        "Coherence", "Completeness", "Naturalness", 
        "Improvement", "Suitability"
    ]].mean()
    print("\nAlternative Diets Average Scores:")
    print(alternative_avg)
else:
    missing_columns = [col for col in ["Improvement", "Suitability"] if col not in evaluation_df.columns]
    print("\nAlternative Diets scores not available:")
    print(f"Missing columns: {missing_columns}")


Evaluating daily diets: 100%|██████████| 40/40 [00:34<00:00,  1.17it/s]
Evaluating alternative diets: 0it [00:00, ?it/s]

Daily Diets Average Scores:
Coherence               3.5750
Completeness            3.3250
Naturalness             3.9625
Nutritional Adequacy    3.0000
Caloric Balance         3.2250
dtype: float64

Alternative Diets scores not available:
Missing columns: ['Improvement', 'Suitability']


In [ ]:
# alternative diets

import openai
import pandas as pd
import re
from tqdm import tqdm

from dotenv import load_dotenv
import os

load_dotenv()


openai.api_key = os.getenv("OPNEAI_API_KEY")

MAX_CONTEXT_LENGTH = 8192

def evaluate_with_gpt4(input_text, model_output, true_output, task_type):

    input_text = input_text[:MAX_CONTEXT_LENGTH]
    model_output = model_output[:MAX_CONTEXT_LENGTH]
    true_output = true_output[:MAX_CONTEXT_LENGTH]

    if task_type == "daily_diets":
        prompt = f"""
        You are tasked with evaluating the quality of a meal recommendation model's responses based on the following metrics:
        1. **Coherence**: Does the model's response logically align with the context provided in the input?
        2. **Completeness**: Does the model's response sufficiently answer the input request?
        3. **Naturalness**: Does the model's response sound fluent and human-like?
        4. **Nutritional Adequacy**: Does the meal response meet the nutritional goals mentioned in the input?
        5. **Caloric Balance**: Are the recommended meals well-balanced in terms of calories?

        **Input**:
        {input_text}

        **Model's Response**:
        {model_output}

        **True Answer**:
        {true_output}

        Please rate each metric on a scale from 1 to 5. 
        Example response format:
        - Coherence: X.X
        - Completeness: X.X
        - Naturalness: X.X
        - Nutritional Adequacy: X.X
        - Caloric Balance: X.X
        """
    elif task_type == "alternative_diet":
        prompt = f"""
        You are tasked with evaluating the quality of a meal recommendation model's responses based on the following metrics:

        1. **Coherence**: Does the model's response logically align with the context provided in the input?
        2. **Completeness**: Does the model's response sufficiently answer the input request?
        3. **Naturalness**: Does the model's response sound fluent and human-like?
        4. **Improvement**: Assume that the recommended alternative meal is an improvement over the previous meal. Evaluate how effectively it builds upon and enhances the previous meal, even if the changes are small or subtle.
        5. **Suitability**: Is the recommended meal suitable for a diabetes patient?

        **Input**:
        {input_text}

        **Model's Response**:
        {model_output}

        **True Answer**:
        {true_output}

        Please rate each metric on a scale from 1 to 5. 
        Example response format:
        - Coherence: X.X
        - Completeness: X.X
        - Naturalness: X.X
        - Improvement: X.X
        - Suitability: X.X
        """
    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-0125",
            messages=[
                {"role": "system", "content": "You are an expert evaluator for meal recommendation models."},
                {"role": "user", "content": prompt}
            ]
        )
        return response["choices"][0]["message"]["content"]
    except Exception as e:
        print("Error with GPT-4 API:", e)
        return None

def extract_scores(evaluation, task_type):
    if evaluation is None:
        if task_type == "daily_diets":
            return {
                "Coherence": 0.0,
                "Completeness": 0.0,
                "Naturalness": 0.0,
                "Nutritional Adequacy": 0.0,
                "Caloric Balance": 0.0
            }
        elif task_type == "alternative_diet":
            return {
                "Coherence": 0.0,
                "Completeness": 0.0,
                "Naturalness": 0.0,
                "Improvement": 0.0,
                "Suitability": 0.0
            }

    scores = {}
    if task_type == "daily_diets":
        metrics = ["Coherence", "Completeness", "Naturalness", "Nutritional Adequacy", "Caloric Balance"]
    elif task_type == "alternative_diet":
        metrics = ["Coherence", "Completeness", "Naturalness", "Improvement", "Suitability"]

    for metric in metrics:
        match = re.search(fr"{metric}: (\d\.\d)", evaluation)
        scores[metric] = float(match.group(1)) if match else 0.0

    return scores

results = []

alternative_df = df[df['task'] == 'alternative_diet']

for _, row in tqdm(alternative_df.iterrows(), total=len(alternative_df), desc="Evaluating alternative diets"):
    evaluation = evaluate_with_gpt4(row['input'], row['model_output'], row['output'], "alternative_diet")
    scores = extract_scores(evaluation, "alternative_diet")
    results.append({**row.to_dict(), **scores})

evaluation_df = pd.DataFrame(results)

if all(col in evaluation_df.columns for col in ["Improvement", "Suitability"]):
    alternative_avg = evaluation_df[[
        "Coherence", "Completeness", "Naturalness", 
        "Improvement", "Suitability"
    ]].mean()
    print("\nAlternative Diets Average Scores:")
    print(alternative_avg)


Evaluating alternative diets:   0%|          | 0/50 [00:00<?, ?it/s]

Evaluating alternative diets: 100%|██████████| 50/50 [00:43<00:00,  1.16it/s]


Alternative Diets Average Scores:
Coherence       4.32
Completeness    3.85
Naturalness     4.02
Improvement     3.27
Suitability     3.90
dtype: float64


In [ ]:
# qa_subjective

import openai
import pandas as pd
import re
from sklearn.metrics import f1_score
from bert_score import score as bert_score
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

from dotenv import load_dotenv
import os

load_dotenv()

openai.api_key = os.getenv("OPENAI_API_KEY")

bleurt_model_name = "Elron/bleurt-large-512"
tokenizer = AutoTokenizer.from_pretrained(bleurt_model_name)
bleurt_model = AutoModelForSequenceClassification.from_pretrained(bleurt_model_name)
bleurt_model.eval()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
bleurt_model = bleurt_model.to(device)

MAX_CONTEXT_LENGTH = 8192

def evaluate_with_gpt4(input_text, model_output, true_output):
    input_text = input_text[:MAX_CONTEXT_LENGTH]
    model_output = model_output[:MAX_CONTEXT_LENGTH]
    true_output = true_output[:MAX_CONTEXT_LENGTH]

    prompt = f"""
    You are tasked with evaluating the quality of a QA model's responses based on the following metrics:
    1. **Coherence**: Does the model's response logically align with the context provided in the input?
    2. **Completeness**: Does the model's response sufficiently answer the question in the input?
    3. **Naturalness**: Does the model's response sound fluent and human-like?

    **Input**:
    {input_text}

    **Model's Response**:
    {model_output}

    **True Answer**:
    {true_output}

    Please rate each metric on a scale from 1 to 5. 
    Example response format:
    - Coherence: X.X
    - Completeness: X.X
    - Naturalness: X.X
    """
    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-0125",
            messages=[
                {"role": "system", "content": "You are an expert evaluator for QA models."},
                {"role": "user", "content": prompt}
            ]
        )
        return response["choices"][0]["message"]["content"]
    except Exception as e:
        print("Error with GPT-4 API:", e)
        return None

def extract_scores(evaluation):
    if evaluation is None:
        return {
            "Coherence": 0.0,
            "Completeness": 0.0,
            "Naturalness": 0.0
        }
    coherence = re.search(r"Coherence: (\d\.\d)", evaluation)
    completeness = re.search(r"Completeness: (\d\.\d)", evaluation)
    naturalness = re.search(r"Naturalness: (\d\.\d)", evaluation)
    
    return {
        "Coherence": float(coherence.group(1)) if coherence else 0.0,
        "Completeness": float(completeness.group(1)) if completeness else 0.0,
        "Naturalness": float(naturalness.group(1)) if naturalness else 0.0
    }

def calculate_concept_f1(y_true, y_pred):
    return f1_score(y_true, y_pred, average="macro")


def calculate_bleurt(y_true, y_pred):
    inputs = tokenizer(y_pred, y_true, return_tensors="pt", padding=True, truncation=True, max_length=512)
    inputs = {key: value.to(device) for key, value in inputs.items()}
    
    with torch.no_grad():
        scores = bleurt_model(**inputs).logits

    if scores.numel() == 1:
        return float(scores.squeeze().item()) 
    return [float(score) for score in scores.squeeze().tolist()] 

def calculate_bleurt_and_bertscore(y_true, y_pred):
    bleurt_score_value = calculate_bleurt(y_true, y_pred)

    _, _, bert_f1 = bert_score(y_pred, y_true, lang="en", rescale_with_baseline=True)

    bert_f1 = [float(score) for score in bert_f1]

    return {
        "BLEURT": bleurt_score_value if isinstance(bleurt_score_value, float) else sum(bleurt_score_value) / len(bleurt_score_value),
        "BERTScore_F1": sum(bert_f1) / len(bert_f1)
    }

def normalize_scores(df, column):

    df[column] = df[column].apply(
        lambda x: float(x) if isinstance(x, torch.Tensor) else x
    )
    min_val = df[column].min()
    max_val = df[column].max()

    df[column] = df[column].apply(
        lambda x: (x - min_val) / (max_val - min_val) if max_val > min_val else 0.5
    )
    return df

qa_df = df[df['task'] == 'qa_subjective']

results = []

for _, row in qa_df.iterrows():
    input_text = row['input']
    model_output = row['model_output']
    true_output = row['output']
    
    evaluation = evaluate_with_gpt4(input_text, model_output, true_output)
    print(f"Evaluation result:\n{evaluation}")
    
    scores = extract_scores(evaluation)
    
    concept_f1 = calculate_concept_f1([true_output], [model_output])
    
    metric_scores = calculate_bleurt_and_bertscore([true_output], [model_output])

    results.append({
        "input": input_text,
        "model_output": model_output,
        "true_output": true_output,
        "evaluation": evaluation,
        "Coherence": scores["Coherence"],
        "Completeness": scores["Completeness"],
        "Naturalness": scores["Naturalness"],
        "Concept_F1": concept_f1,
        "BLEURT": metric_scores["BLEURT"],
        "BERTScore_F1": metric_scores["BERTScore_F1"]
    })

evaluation_df = pd.DataFrame(results)

evaluation_df = normalize_scores(evaluation_df, "BLEURT")
evaluation_df = normalize_scores(evaluation_df, "BERTScore_F1")

average_scores = evaluation_df[["Coherence", "Completeness", "Naturalness", "Concept_F1", "BLEURT", "BERTScore_F1"]].mean()

print("평균 점수:")
print(average_scores)

Evaluation result:
- Coherence: 3.5
- Completeness: 3.0
- Naturalness: 2.5


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Evaluation result:
- Coherence: 1.0
- Completeness: 1.0
- Naturalness: 3.5


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Evaluation result:
- Coherence: 3.5
- Completeness: 3.0
- Naturalness: 4.0


...

Evaluation result:
- Coherence: 3.5
- Completeness: 4.0
- Naturalness: 3.5


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Evaluation result:
- Coherence: 1.0
- Completeness: 2.0
- Naturalness: 4.0


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


평균 점수:
Coherence       3.310000
Completeness    2.920000
Naturalness     3.180000
Concept_F1      0.000000
BLEURT          0.515577
BERTScore_F1    0.639133
dtype: float64


In [9]:
## nutri score, HEI

df[df['task']=='daily_diets'].head(2)

,dataset,split_data,task,instruction,input,output,__index_level_0__,model_output,output_parsed,model_output_parsed
430,diabetes_food_hub,test,daily_diets,Recommend a daily diet based on the given nutr...,Ensure the daily carbohydrate intake does not ...,"{""Breakfast"": ""Chicken and Cucumber Lettuce Wr...",32166,"{""Breakfast"": ""Turkey Sausage and Egg Casserol...",None,None
431,diabetes_food_hub,test,daily_diets,Recommend a daily diet that includes a specifi...,Create a diet that includes Parmesan cheese(gr...,"{""Breakfast"": ""Chicken and Cucumber Lettuce Wr...",32167,"{""Breakfast"": ""Turkey Meatball \u201cWonton\u2...",None,None


In [10]:
df[df['task']=='daily_diets'].iloc[1]['model_output']

'{"Breakfast": "Turkey Meatball \\u201cWonton\\u201d Soup with Bok Choy & Carrots", "Lunch": "Peach & Cream Smoothie", "Dinner": "Curried Chickpea Stew with Roasted Vegetables"}'

In [12]:
import pandas as pd

dfh = pd.read_csv("/data/jaesung/llm_for_diabetes/src/data/data2_daily_diets/diabetes_food_hub_new_nutri_facts.csv")
dfh.head(2)

,title,description,prep_time,cook_time,servings,steps,tags,nutrition_facts,ingredients
0,Raspberry Swirl Frozen Yogurt Bark,Raspberry Swirl Frozen Yogurt Bark: Dive into ...,10 min,4 hr,6 Servings,['Cover a freezer-safe tray with parchment pap...,"['Kid Friendly', 'Vegetarian', 'Dessert', 'Sna...","{'Servings': '6 Servings', 'Serving Size': '1 ...","[{'label': 'Plain Nonfat Greek yogurt', 'us_me..."
1,Maple-Pumpkin Spice Oatmeal Cookies,Description not found,10 min,25 min,14 Servings,['Preheat the oven to 350 degrees F. Line two ...,"['Kid Friendly', 'Vegetarian', 'Snacks', 'Glut...","{'Servings': '14 Servings', 'Serving Size': '1...","[{'label': 'old-fashioned rolled oats', 'us_me..."


In [13]:
print(dfh['nutrition_facts'].iloc[0])

{'Servings': '6 Servings', 'Serving Size': '1 slice (4×4 inch square)', 'Amount per Serving': {'Calories': '70', 'Total Fat': {'Amount': '1g', 'Saturated Fat': '1g', 'Trans Fat': '0g'}, 'Cholesterol': '5mg', 'Sodium': '30mg', 'Total Carbohydrates': {'Amount': '19g', 'Dietary Fiber': '1g', 'Total Sugars': '4g', 'Added Sugars': '0g'}, 'Protein': '8g', 'Potassium': '140mg', 'Phosphorus': None}}


In [14]:
print(type(dfh['nutrition_facts'].iloc[0]))

<class 'str'>


In [15]:
dfh['ingredients'].iloc[0]

"[{'label': 'Plain Nonfat Greek yogurt', 'us_measure': '2 cup', 'metric_measure': '473 ml'}, {'label': 'pure vanilla extract', 'us_measure': '1 tsp', 'metric_measure': '5 g'}, {'label': 'Splenda® Multi-Use Syrup(divided)', 'us_measure': '1/3 cup', 'metric_measure': '78 ml'}, {'label': 'fresh or frozen raspberries', 'us_measure': '1/2 cup', 'metric_measure': '118 ml'}, {'label': 'Fresh Blueberries', 'us_measure': '1/2 cup', 'metric_measure': '118 ml'}, {'label': 'unsweetened coconut flakes', 'us_measure': '1 tbsp', 'metric_measure': '15 ml'}]"

In [16]:
dfh.columns

Index(['title', 'description', 'prep_time', 'cook_time', 'servings', 'steps',
       'tags', 'nutrition_facts', 'ingredients'],
      dtype='object')

In [32]:
# data2 - nutri score

import ast
import pandas as pd
import json
import re
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

def extract_numeric_value(value):
    try:
        if isinstance(value, str):
            match = re.search(r"(\d+(\.\d+)?)", value)
            if match:
                return float(match.group(1))
        elif isinstance(value, (int, float)):
            return float(value)
    except Exception as e:
        print(f"Error in extract_numeric_value: {e}, value: {value}")
    return 0.0

def is_valid_meal_structure(json_string):
    try:
        data = json.loads(json_string)
        return all(key in data for key in ['Breakfast', 'Lunch', 'Dinner'])
    except (json.JSONDecodeError, TypeError):
        return False

def find_most_similar_row(title, dfh):
    try:
        dfh['title'] = dfh['title'].fillna('')  # NaN 처리
        vectorizer = TfidfVectorizer()
        tfidf_matrix = vectorizer.fit_transform(dfh['title'])
        input_vector = vectorizer.transform([title])
        similarities = cosine_similarity(input_vector, tfidf_matrix)
        most_similar_idx = similarities.argmax()
        return dfh.iloc[most_similar_idx]
    except Exception as e:
        print(f"Error in find_most_similar_row: {e}, title: {title}")
        return None

def extract_nested_value(data, keys, default=0):
    try:
        for key in keys:
            if isinstance(data, dict):
                data = data.get(key, {})
            else:
                return default
        return extract_numeric_value(data)
    except Exception as e:
        print(f"Error in extract_nested_value: {e}, keys: {keys}, data: {data}")
        return default

def calculate_nutri_score(nutrition_facts):
    try:
        if isinstance(nutrition_facts, str):
            nutrition_facts = ast.literal_eval(nutrition_facts)

        energy = extract_nested_value(nutrition_facts, ['Amount per Serving', 'Calories'])
        saturated_fat = extract_nested_value(nutrition_facts, ['Amount per Serving', 'Total Fat', 'Amount'])
        sugar = extract_nested_value(nutrition_facts, ['Amount per Serving', 'Total Carbohydrates', 'Total Sugars'])
        sodium = extract_nested_value(nutrition_facts, ['Amount per Serving', 'Sodium'])
        fiber = extract_nested_value(nutrition_facts, ['Amount per Serving', 'Total Carbohydrates', 'Dietary Fiber'])
        protein = extract_nested_value(nutrition_facts, ['Amount per Serving', 'Protein'])

        # 불리한 점수 계산
        energy_points = min(energy / 80, 800)
        saturated_fat_points = min(saturated_fat / 1, 10)
        sugar_points = min(sugar / 4.5, 45)
        sodium_points = min(sodium / 90, 900)

        unfavorable_points = energy_points + saturated_fat_points + sugar_points + sodium_points

        # 유리한 점수 계산
        fiber_points = min(fiber / 0.7, 3.5)
        protein_points = min(protein / 1.6, 8.0)
        fruit_veg_points = 0  # 과일/야채 정보가 없으므로 기본값 사용

        favorable_points = fiber_points + protein_points + fruit_veg_points

        # 최종 Nutri-Score 계산
        total_score = unfavorable_points - favorable_points
        return total_score
    except Exception as e:
        print(f"Error in calculate_nutri_score: {e}, nutrition_facts: {nutrition_facts}")
        return None

def get_nutri_score_grade(score):
    """
    Nutri-Score 등급 계산.
    """
    if score <= -1:
        return "A"
    elif score <= 2:
        return "B"
    elif score <= 10:
        return "C"
    elif score <= 18:
        return "D"
    else:
        return "E"

def calculate_meal_nutri_score(meal_data, dfh):
    meal_scores = {}

    for meal, title in meal_data.items():
        matched_row = find_most_similar_row(title, dfh)
        if matched_row is None:
            continue

        nutrition_facts = matched_row['nutrition_facts']
        score = calculate_nutri_score(nutrition_facts)
        grade = get_nutri_score_grade(score)

        meal_scores[meal] = {
            'score': score,
            'grade': grade
        }

    return meal_scores

def calculate_scores_with_comparison(df, dfh):
    """
    output 및 model_output Nutri-Score를 계산하고 비교.
    """
    results = []
    output_scores_list = []
    model_output_scores_list = []

    for idx, row in df.iterrows():
        try:
            if is_valid_meal_structure(row.get('output', '')):
                output_data = json.loads(row['output'])
                output_scores = calculate_meal_nutri_score(output_data, dfh)
                output_avg_score = sum([meal['score'] for meal in output_scores.values() if meal['score'] is not None]) / len(output_scores)
                output_scores_list.append(output_avg_score)
            else:
                output_avg_score = None

            if is_valid_meal_structure(row.get('model_output', '')):
                model_output_data = json.loads(row['model_output'])
                model_output_scores = calculate_meal_nutri_score(model_output_data, dfh)
                model_output_avg_score = sum([meal['score'] for meal in model_output_scores.values() if meal['score'] is not None]) / len(model_output_scores)
                model_output_scores_list.append(model_output_avg_score)
            else:
                model_output_avg_score = None

            results.append({
                'row_index': idx,
                'output_avg_score': output_avg_score,
                'model_output_avg_score': model_output_avg_score
            })

        except Exception as e:
            print(f"Error processing row {idx}: {e}")
            results.append({
                'row_index': idx,
                'output_avg_score': None,
                'model_output_avg_score': None
            })

    final_output_avg = sum(output_scores_list) / len(output_scores_list) if output_scores_list else None
    final_model_output_avg = sum(model_output_scores_list) / len(model_output_scores_list) if model_output_scores_list else None

    print(f"Output 평균 Nutri-Score: {final_output_avg}")
    print(f"Model Output 평균 Nutri-Score: {final_model_output_avg}")

    return results

filtered_df = df[df['task'] == 'daily_diets'] 
results = calculate_scores_with_comparison(filtered_df, dfh)

for result in results:
    print(result)


Output 평균 Nutri-Score: 7.549596560846561
Model Output 평균 Nutri-Score: 7.859275793650793
{'row_index': 430, 'output_avg_score': 9.847222222222221, 'model_output_avg_score': 6.273809523809523}
{'row_index': 431, 'output_avg_score': 9.847222222222221, 'model_output_avg_score': 7.722222222222221}
{'row_index': 432, 'output_avg_score': 9.847222222222221, 'model_output_avg_score': 9.231481481481483}
{'row_index': 433, 'output_avg_score': 9.847222222222221, 'model_output_avg_score': 11.056878306878305}
{'row_index': 434, 'output_avg_score': None, 'model_output_avg_score': None}
{'row_index': 435, 'output_avg_score': 9.032407407407407, 'model_output_avg_score': 6.504629629629631}
{'row_index': 436, 'output_avg_score': 9.032407407407407, 'model_output_avg_score': 6.5}
{'row_index': 437, 'output_avg_score': 9.032407407407407, 'model_output_avg_score': 8.041666666666666}
{'row_index': 438, 'output_avg_score': 9.032407407407407, 'model_output_avg_score': 5.951719576719576}
{'row_index': 439, 'outp

In [30]:
df[df['task'] == 'alternative_diet'].head(2)

,dataset,split_data,task,instruction,input,output,__index_level_0__,model_output,output_parsed,model_output_parsed
480,diabetes_food_hub,test,alternative_diet,"Based on the previous meal, suggest the next m...",Flax seeds,"Orange, Asparagus, and Avocado Salad is recomm...",35166,Greek Bean Salad is recommended. The reason is...,None,None
481,diabetes_food_hub,test,alternative_diet,"Based on the previous meal, suggest the next m...","Lettuce, arugula, raw",Budget-Friendly Vegetable Stew with Whole Whea...,35167,Grilled Chicken Sausage with Zucchini and Pepp...,None,None


In [35]:
df[df['task']=='alternative_diet']['model_output'].iloc[2]

"Balsamic Chicken & Peppers is recommended. The reason is 'Balsamic Chicken & Peppers' complements 'Salmon, baked or broiled, made with oil' as it provides a different protein source, additional vitamins, and lower fat content. However, caution should be taken with the sodium content in the dish."

In [36]:
# nutri score - alternative diet

import ast
import pandas as pd
import json
import re
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

def extract_numeric_value(value):
    try:
        if isinstance(value, str):
            match = re.search(r"(\d+(\.\d+)?)", value)
            if match:
                return float(match.group(1))
        elif isinstance(value, (int, float)):
            return float(value)
    except Exception as e:
        print(f"Error in extract_numeric_value: {e}, value: {value}")
    return 0.0

def is_valid_meal_structure(json_string):
    try:
        data = json.loads(json_string)
        return isinstance(data, dict)
    except (json.JSONDecodeError, TypeError):
        return False

def find_most_similar_row(title, dfh):
    try:
        dfh['title'] = dfh['title'].fillna('')  # NaN 처리
        vectorizer = TfidfVectorizer()
        tfidf_matrix = vectorizer.fit_transform(dfh['title'])
        input_vector = vectorizer.transform([title])
        similarities = cosine_similarity(input_vector, tfidf_matrix)
        most_similar_idx = similarities.argmax()
        return dfh.iloc[most_similar_idx]
    except Exception as e:
        print(f"Error in find_most_similar_row: {e}, title: {title}")
        return None

def extract_nested_value(data, keys, default=0):
    try:
        for key in keys:
            if isinstance(data, dict):
                data = data.get(key, {})
            else:
                return default
        return extract_numeric_value(data)
    except Exception as e:
        print(f"Error in extract_nested_value: {e}, keys: {keys}, data: {data}")
        return default

def calculate_nutri_score(nutrition_facts):
    try:
        if isinstance(nutrition_facts, str):
            nutrition_facts = ast.literal_eval(nutrition_facts)

        energy = extract_nested_value(nutrition_facts, ['Amount per Serving', 'Calories'])
        saturated_fat = extract_nested_value(nutrition_facts, ['Amount per Serving', 'Total Fat', 'Amount'])
        sugar = extract_nested_value(nutrition_facts, ['Amount per Serving', 'Total Carbohydrates', 'Total Sugars'])
        sodium = extract_nested_value(nutrition_facts, ['Amount per Serving', 'Sodium'])
        fiber = extract_nested_value(nutrition_facts, ['Amount per Serving', 'Total Carbohydrates', 'Dietary Fiber'])
        protein = extract_nested_value(nutrition_facts, ['Amount per Serving', 'Protein'])

        energy_points = min(energy / 80, 800)
        saturated_fat_points = min(saturated_fat / 1, 10)
        sugar_points = min(sugar / 4.5, 45)
        sodium_points = min(sodium / 90, 900)

        unfavorable_points = energy_points + saturated_fat_points + sugar_points + sodium_points

        fiber_points = min(fiber / 0.7, 3.5)
        protein_points = min(protein / 1.6, 8.0)
        fruit_veg_points = 0  # 과일/야채 정보가 없으므로 기본값 사용

        favorable_points = fiber_points + protein_points + fruit_veg_points

        total_score = unfavorable_points - favorable_points
        return total_score
    except Exception as e:
        print(f"Error in calculate_nutri_score: {e}, nutrition_facts: {nutrition_facts}")
        return None

def get_nutri_score_grade(score):
    if score <= -1:
        return "A"
    elif score <= 2:
        return "B"
    elif score <= 10:
        return "C"
    elif score <= 18:
        return "D"
    else:
        return "E"

def extract_recommended_text(text):
    """
    'is recommended' 앞의 텍스트를 추출합니다.
    """
    try:
        if isinstance(text, str):
            recommended_text = text.split('is recommended')[0].strip()
            return recommended_text
        return None
    except Exception as e:
        print(f"Error in extract_recommended_text: {e}, text: {text}")
        return None

def calculate_scores_with_comparison_no_meals(df, dfh):
    """
    output 및 model_output에서 'is recommended' 앞의 텍스트를 추출하여 Nutri-Score를 계산하고 비교합니다.
    """
    results = []
    output_scores_list = []
    model_output_scores_list = []

    for idx, row in df.iterrows():
        try:
            output_text = extract_recommended_text(row.get('output', ''))
            if output_text:
                matched_row = find_most_similar_row(output_text, dfh)
                if matched_row is not None:
                    nutrition_facts = matched_row['nutrition_facts']
                    output_score = calculate_nutri_score(nutrition_facts)
                    output_scores_list.append(output_score)
                else:
                    output_score = None
            else:
                output_score = None

            model_output_text = extract_recommended_text(row.get('model_output', ''))
            if model_output_text:
                matched_row = find_most_similar_row(model_output_text, dfh)
                if matched_row is not None:
                    nutrition_facts = matched_row['nutrition_facts']
                    model_output_score = calculate_nutri_score(nutrition_facts)
                    model_output_scores_list.append(model_output_score)
                else:
                    model_output_score = None
            else:
                model_output_score = None

            results.append({
                'row_index': idx,
                'output_score': output_score,
                'model_output_score': model_output_score
            })

        except Exception as e:
            print(f"Error processing row {idx}: {e}")
            results.append({
                'row_index': idx,
                'output_score': None,
                'model_output_score': None
            })

    final_output_avg = sum(output_scores_list) / len(output_scores_list) if output_scores_list else None
    final_model_output_avg = sum(model_output_scores_list) / len(model_output_scores_list) if model_output_scores_list else None

    print(f"Output 평균 Nutri-Score: {final_output_avg}")
    print(f"Model Output 평균 Nutri-Score: {final_model_output_avg}")

    return results

filtered_df = df[df['task'] == 'alternative_diet']
results = calculate_scores_with_comparison_no_meals(filtered_df, dfh)

for result in results:
    print(result)


Output 평균 Nutri-Score: 4.111186507936508
Model Output 평균 Nutri-Score: 5.538075396825397
{'row_index': 480, 'output_score': 6.180555555555555, 'model_output_score': 6.416666666666666}
{'row_index': 481, 'output_score': 2.2361111111111107, 'model_output_score': 6.392857142857142}
{'row_index': 482, 'output_score': 6.392857142857142, 'model_output_score': 5.099206349206348}
{'row_index': 483, 'output_score': 2.9722222222222214, 'model_output_score': 6.416666666666666}
{'row_index': 484, 'output_score': 6.392857142857142, 'model_output_score': 4.781746031746032}
{'row_index': 485, 'output_score': 6.392857142857142, 'model_output_score': 8.48611111111111}
{'row_index': 486, 'output_score': -0.27777777777777857, 'model_output_score': 3.113095238095238}
{'row_index': 487, 'output_score': 2.8055555555555554, 'model_output_score': 6.416666666666666}
{'row_index': 488, 'output_score': 5.729166666666666, 'model_output_score': 8.48611111111111}
{'row_index': 489, 'output_score': 0.7083333333333339